In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/4.page.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Make No'] = df_input['Make No'].astype(int)
df_input['Year'] = df_input['Year'].astype(int)
df_input['Model No'] = df_input['Model No'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：6329



,Make No,Make,Year,Model No,Model,Model Text,Model Url,Page
0,1,Acura,2024,1,Integra,2024 Acura Integra TPMS,https://www.tpms.com/2024-Acura-Integra-TPMS-s...,1
1,1,Acura,2024,2,MDX,2024 Acura MDX TPMS,https://www.tpms.com/2024-Acura-MDX-TPMS-s/113...,1
2,1,Acura,2024,3,RDX,2024 Acura RDX TPMS,https://www.tpms.com/2024-Acura-RDX-TPMS-s/113...,1
3,1,Acura,2024,4,TLX,2024 Acura TLX TPMS,https://www.tpms.com/2024-Acura-TLX-TPMS-s/113...,1
4,1,Acura,2023,1,Integra,2023 Acura Integra TPMS,https://www.tpms.com/2023-Acura-Integra-TPMS-s...,1
...,...,...,...,...,...,...,...,...
6324,53,Volvo,2004,2,S60,2004 Volvo S60 TPMS,https://www.tpms.com/2004-Volvo-S60-TPMS-s/490...,1
6325,53,Volvo,2004,3,S80,2004 Volvo S80 TPMS,https://www.tpms.com/2004-Volvo-S80-TPMS-s/490...,1
6326,53,Volvo,2004,4,V70,2004 Volvo V70 TPMS,https://www.tpms.com/2004-Volvo-V70-TPMS-s/490...,1
6327,53,Volvo,2004,5,XC70,2004 Volvo XC70 TPMS,https://www.tpms.com/2004-Volvo-XC70-TPMS-s/49...,1


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Make No                                                       1
 Make                                                      Acura
 Year                                                       2024
 Model No                                                      1
 Model                                                   Integra
 Model Text                              2024 Acura Integra TPMS
 Model Url     https://www.tpms.com/2024-Acura-Integra-TPMS-s...
 Page                                                          1
 Name: 0, dtype: object)

In [4]:
import time

list_dict_correct_temp = []
for page in range(1, crawler_series['Page'] + 1):

    count_retry = 0
    while True:
        count_retry += 1
        print(page, count_retry)
        try:
            url_request = f'''{crawler_series['Model Url']}?searching=Y&sort=7&show=300&page={page}'''

            resp = requests.get(url_request,
                                headers=get_header(),
                                proxies=get_proxy(),
                                timeout=(5, 15))

            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                if html.xpath(f'''//b/a[@href="{crawler_series['Model Url']}"]/text()''')[0] == crawler_series['Model Text']:
                    if page == 0:
                        with open('./url.html', 'w', encoding='UTF-8') as file:
                            file.write(str(soup))
                    break
        except KeyboardInterrupt:
            break
        except:
            time.sleep(3)
            continue

    # = = = = = = = = = = = = = = =

    list_url = [href.strip() for href in html.xpath('//div[@class="v-product"]/a[@class="v-product__title productnamecolor colors_productname"]/@href')]

    # = = = = = = = = = = = = = = =

    if not list_url:
        print('fuck')

    # = = = = = = = = = = = = = = =

    list_dict_correct_temp.extend([{'Make No': crawler_series['Make No'],
                                    'Make': crawler_series['Make'],
                                    'Year': crawler_series['Year'],
                                    'Model No': crawler_series['Model No'],
                                    'Model': crawler_series['Model'],
                                    'Page': crawler_series['Page'],
                                    'No': i + 1,
                                    'Url': url} for i, url in enumerate(list_url)])

list_dict_correct_temp

1 1


[{'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 1,
  'Url': 'https://www.tpms.com/Acura-Huf-IntelliSens-TPMS-Sensor-p/uvc-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 2,
  'Url': 'https://www.tpms.com/Acura-Huf-IntelliSens-TPMS-Sensor-p/uvs-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 3,
  'Url': 'https://www.tpms.com/Acura-TSS1-TPMS-Sensor-p/tss1r-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 4,
  'Url': 'https://www.tpms.com/Acura-TSS1-TPMS-Sensor-p/tss-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 5,
  'Url': 'https://www.tpms.com/Acura-Upro-TPMS-Sensor-p/up

In [5]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 1,
  'Url': 'https://www.tpms.com/Acura-Huf-IntelliSens-TPMS-Sensor-p/uvc-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 2,
  'Url': 'https://www.tpms.com/Acura-Huf-IntelliSens-TPMS-Sensor-p/uvs-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 3,
  'Url': 'https://www.tpms.com/Acura-TSS1-TPMS-Sensor-p/tss1r-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 4,
  'Url': 'https://www.tpms.com/Acura-TSS1-TPMS-Sensor-p/tss-42753-t3r-a04-acura.htm'},
 {'Make No': 1,
  'Make': 'Acura',
  'Year': 2024,
  'Model No': 1,
  'Model': 'Integra',
  'Page': 1,
  'No': 5,
  'Url': 'https://www.tpms.com/Acura-Upro-TPMS-Sensor-p/up

In [6]:
crawler_status = True

crawler_status

True

In [7]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[Make No                                                        1
 Make                                                       Acura
 Year                                                        2024
 Model No                                                       1
 Model                                                    Integra
 Model Text                               2024 Acura Integra TPMS
 Model Url      https://www.tpms.com/2024-Acura-Integra-TPMS-s...
 Page                                                           1
 Request_Url    https://www.tpms.com/2024-Acura-Integra-TPMS-s...
 Name: 0, dtype: object]

In [8]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Make No', 'Year', 'Model No', 'Page', 'No'],
                                                             ascending=[True, False, True, True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_url.xlsx', index=False)

df_correct

,Make No,Make,Year,Model No,Model,Page,No,Url
0,1,Acura,2024,1,Integra,1,1,https://www.tpms.com/Acura-Huf-IntelliSens-TPM...
1,1,Acura,2024,1,Integra,1,2,https://www.tpms.com/Acura-Huf-IntelliSens-TPM...
2,1,Acura,2024,1,Integra,1,3,https://www.tpms.com/Acura-TSS1-TPMS-Sensor-p/...
3,1,Acura,2024,1,Integra,1,4,https://www.tpms.com/Acura-TSS1-TPMS-Sensor-p/...
4,1,Acura,2024,1,Integra,1,5,https://www.tpms.com/Acura-Upro-TPMS-Sensor-p/...
5,1,Acura,2024,1,Integra,1,6,https://www.tpms.com/Acura-Upro-TPMS-Sensor-p/...


In [9]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Make No', 'Year', 'Model No', 'Page'],
                                                           ascending=[True, False, True, True],
                                                           ignore_index=True)
    df_error.to_excel('./test_url_error.xlsx', index=False)

df_error

,Make No,Make,Year,Model No,Model,Model Text,Model Url,Page,Request_Url
0,1,Acura,2024,1,Integra,2024 Acura Integra TPMS,https://www.tpms.com/2024-Acura-Integra-TPMS-s...,1,https://www.tpms.com/2024-Acura-Integra-TPMS-s...
